In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

   **MOBILE PRICE CLASSIFICATION DATASET**

Bu çalışma sayfasında Mobile Price Classification datası ile ilgili yeni bir çözüm örneğini paylaşıyorum. Konu hakkındaki yorumlarınız benim için önemlidir.

Bazı kısımlar da görüş eksikliği veya dolaylı yoldan gidilmiş olabilir yeni olan tüm önerilere açığım umarım bu örnek çalışma sizin için faydalı olur.


Merhaba bu çalışma da "Mobile Price Classification" datası ile çeşitli makine öğrenmesi algoritmaları kurup bu algoritmaları iyileştirmeye çalışacağız. Aynı zamanda data da bulunan değişkenleri inceleyerek bazı değişimler yapıp modelimizi daha iyi açıklayacı hale getirmeye çalışacağız.

Önce datamızı tanıyarak başlayalım...

Mobile Price Classification datasında bizlere 20 farklı değişken veriyor. Nedir bu değişkenler?

1. Battery Power (Telefona ait batarya kapasitesini göstern deişken)
2. Blue (Bluetooth özelliğinin olup olmadığını gösteren değişken)
3. Clock Speed (İşlemci hızını gösteren değişken)
4. Dual Sim (Çift sim kart olup olmadığını gösteren değişken)
5. FC (Ön kamera mp değeri)
6. 4G (Telefonun 4G bağlantısı olup olamdığını gösteren değişken)
7. İnt Memory (Bellek)
8. Mobile_Wt (Telefonun kalınlığını gösteren değer)
9. n_cores (İşlemci sayısı)
10. PC (Arka kamera mp değeri)
11. px_height (Çözünürlük)
12. px_width (Çözünürlük)
13. Ram (ram)
14. sc_h (Ekran yüksekliği)
15. sc_w (Ekran genişliği)
16. talk_time (Konuşma süresi)
17. three_g (3G özelliğinin olup olmadığını gösteren değer)
18. touch_screen (Dokunmatik olup olamdığını gösteren değer)
19. wifi (kablosuz internetin olup olmadığını gösteren değer)
20. price_range (fiyat aralığı -hedef değişken)

Amacımız yukarıdaki değişkenleri kullanarak hedef değişkenimiz olan price_range değerini yüksek orandan doğru tahmin etmeye çalışmak.

Başlayalım.


İlk olarak başlangıç kütüphanelerini ve train-test datasını yükleyerek başlayalım.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
test = pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")

Verinin ilk 5 satırı, veriye ait veri tipleri, boş veri olup olamdığı ve basit istatistik değerlerini inceleyelim.

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()/len(train)

In [ ]:
train.describe()

Değişkenlerin veri tipleri int ve float herhangi encode işlemi yapmamıza gerek yok aynı zaman da veri de boş satırlarda yok!

Şimdi hedef değişken olan price_range içerisinde kaç farklı sınıf var ve bu sınıflara ait ne kadar değer var bunlara bakalım.

In [ ]:
train["price_range"].value_counts()

0, 1, 2 ve 3 diye 4 farklı sınıfımız var biz bunları doğru tahmin etmeye çalışcağız. İşin güzel yanı bu dataset örnek bir dataset olduğu için sınıflar eşit miktarlarda eğer düzensiz olarak dağılsalardı model ilk aşamada daha fazla sayıda örnek bulunduran sınıfa yanlı davranabilirdi. Sınıfların çok düzensiz dağıldığı örnekler resample işlemi faydalı olabilir. Örneğin hedef değişken "evet" ve "hayır" şeklinde oluşan 2 farklı sınıf olsun ve "hayır" cevaplarının miktarı "evet"'lerin 10 katı, 100 katı hatta 1000 katı olduğunu düşünelim bu tip proplemlerde modeller genellikle hayır cevabına yanlı olacaktırlar. Bu problemi çözmek için eğer elinizde yeteri kadar data olduğunu düşünüyorsanız "hayır" ları "evet" lerin sayısını inderecek kadar satırı datanızdan çıkararak düzenleyebilirsiniz yada "evet" leri "hayır" ların sayısına getirecek kadar çoğaltabilirsiniz. Örnek çalışma için --> [https://www.kaggle.com/alpertemel/fraud-data-93-accuracy](http://)

**price_range değişkeni ile en fazla kolerasyona sahip olan değişkenleri bulmak**

In [ ]:
corr = train.corr().abs()
n_most_correlated = 10
most_correlated_feature = corr["price_range"].sort_values(ascending = False)[:n_most_correlated].drop("price_range")
most_correlated_feature_name = most_correlated_feature.index.values

f, ax = plt.subplots(figsize = (15, 6))
plt.xticks(rotation = "90")
sns.barplot(x = most_correlated_feature_name, y = most_correlated_feature)
plt.title("En fazle kolerasyona sahip değişkenler")

En yüksek kolerasyona sahip değişken "ram". Ram neredeyse tek başına price_range değerini belirleyebilecek oranda güçlü.

En fazla kolerasyona sahip 10 değişkeni gördük şimdi de tüm değişkenlerin birbirleyle olan kolerasyonlarını bakalım.

In [ ]:
corr2 = train.corr()
f, ax = plt.subplots(figsize = (15, 6))
sns.heatmap(corr2)

Görüldüğü gibi price_range ile ram'dan başka iyi bir ilişkisi olan değişken yok. Çoğu 0 değerine yakın yani hiçbir açıklayıcılıkları yok. Aynı zaman da bir iki örnek hariç (ön ve arka kamera, 3G ve 4G gibi) değişkenlerin birbirleri ile de bir ilişkisi yok fakat yine de bizim datamızdalar. 

Bunu daha net görebilmek için price_range ile en fazla ilişkiye sahip 10 değişkeni alıp bir pair plot çizdireceğim.

In [ ]:
most_correlated_feature2 = corr["price_range"].sort_values(ascending = False)[:10]
fair = most_correlated_feature2.index

train2 = pd.DataFrame()

for i in fair:
    train2 = pd.concat([train2, train[i]], axis = 1)

    
sns.pairplot(data = train2, hue = "price_range")

Yukarıdaki grafikler de price_range değişkeninin ram hariç diğer değişkenlerde düzensiz, ratgele dağıldı gözüküyor ve bunlar en yüksek kolerasyona sahip 10 değişken. Belki de price_range'i açıklamak için bu değişkenlerin bir çoğuna ihtiyacımız yok ve belki de kuracağımız modelde fazlalık olup daha düşük bir skor elde etmemize neden olacaklar. 

Denemek için şöyle bir yöntem izleyeceğim;

Önce hiçbir işlem yapmadan train ve test setini hazılayıp bir model kuracağım ardından feature selection kullanarak çıkarmam gereken değişkenleri bulup çıkardıktan sonra hiçbir değişik yapmadan aynı modeli tekrar kuracağım.

** feature selection --> 

"Özellik seçimi (feature selection), orijinal veri setini temsil edebilecek en iyi altkümenin seçimi olarak tanımlanmaktadır. Özellik seçimi (diğer adıyla nitelik seçimi veya değişken seçimi), kullanılan algoritmaya göre özellikleri değerlendirerek veri setindeki n adet özellik arasından en iyi k adet özelliği seçme işlemidir."

Özellik Seçim Yöntemleri ve Yeni Bir Yaklaşım
Hüseyin BUDAK

Daha ayrıntılı bilgi --> [https://dergipark.org.tr/en/download/article-file/552933](http://)

In [ ]:
x = train.iloc[:, 0:20]
y = train.iloc[:, 20:21]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 23)

#datanın train ve test setlerine bölümü.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_tahmin = rf.predict(x_test)

print("Accuracy Score: ", accuracy_score(y_test, rf_tahmin))

İlk sonucumuzun değeri %75.

Şimdi feature selection yaparak yine random forest modeli kuralım.

In [ ]:
import statsmodels.api as sm
X_1 = sm.add_constant(x)

model = sm.OLS(y,X_1).fit()
model.pvalues

cols2 = list(x.columns)
pmax = 1
while (len(cols2)>0):
    p= []
    X_1 = x[cols2]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols2)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols2.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols2
print(selected_features_BE)
len(selected_features_BE)


train2 = pd.DataFrame()

for i in selected_features_BE:
    train2 = pd.concat([train2, train[i]], axis = 1)
    
train2 = pd.concat([train2, train["price_range"]], axis = 1)

x2 = train2.iloc[:, 0:6]
y2 = train2.iloc[:, 6:7]

X_train, X_test, Y_train, Y_test = train_test_split(x2, y2, test_size =0.33, random_state =34)

rf2 = RandomForestClassifier()
rf2.fit(X_train, Y_train)
rf2_tahmin = rf2.predict(X_test)

print("FEATURE SELECTION İLE ACCURACY SCORE: ",accuracy_score(Y_test, rf2_tahmin))

Feature Selection ile model %86 başarı sağladı. Eski halinden 10 puan daha iyi bu oldukça güzel bir fark.

Modele feature selection ile devam etmek daha iyi.

Şimdi sırasıyla knn ve svm modellerini deneyelim ve bu modelleri tune ederek nihai sonuca ulaşalım.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
knn_tahmin = knn.predict(X_test)

accuracy_score(Y_test, knn_tahmin)

In [ ]:
from sklearn.model_selection import GridSearchCV

knn_params = {
        "n_neighbors" : np.arange(2, 10, 1),
        "weights" : ["uniform", "distance"],
        "leaf_size" : (30, 40, 50, 20),
        "p" : (1, 2)
        }

knn_ = KNeighborsClassifier()

knn_grid = GridSearchCV(knn_, knn_params, cv = 10, n_jobs = -1, verbose = 2)
knn_grid.fit(X_train, Y_train)
knn_grid.best_params_

knn_tune = KNeighborsClassifier(leaf_size = 30, n_neighbors = 9, p = 2, weights = "uniform")
knn_tune.fit(X_train, Y_train)
knn_tune_tahmin = knn_tune.predict(X_test)

accuracy_score(Y_test, knn_tune_tahmin)

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel = "linear")
svm.fit(X_train, Y_train)
svm_tahmin = svm.predict(X_test)

accuracy_score(Y_test, svm_tahmin)

En yüksek skoru %97 ile SVM çıkarmış oldu.

Umarım bu çalışma sizin için faydalı olmuştur.

Tekrar görüşmek üzere.

Alper Temel.